In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier

from sk_modelcurves.learning_curve import draw_learning_curve

import matplotlib.pyplot as plt
%matplotlib inline

### Describe the content of the dataset and its goals
The dataset is made up of all numerical data of a subset of medical data relating to patient diagnosis for diabetes according to WHO criteria.

The goal of the dataset is to try and determine possible feature importance to forecast diabetes for this specific population, females at least 21 years old of Pima Indian heritage.

There are some missing values for some of the data variables.

In [17]:
col_names = ['times_pregnant','glucose_concentration','blood_pressure','skin_fold_thickness','serum_insulin','bmi','pedigree','age','class']
na={"glucose_concentration" : "0", "blood_pressure" : "0", "skin_fold_thickness" : "0", "serum_insulin" : "0", "bmi" : "0"}
pima_data = pd.read_csv('pima-indians-diabetes/pima-indians-diabetes.data', names=col_names, na_values=na)

### Describe the features and formulate a hypothesis on which might be relevant in predicting diabetes

After doing basic data exploration, looking at the correlation values and the counts it seems to me that glucose_concentration and bmi are the most helpful features in predicting diabetes, at least according to this dataset, on the surface you would think that skin_fold_thickness would be closely related to bmi but we have a lot of null values there and insulin would also be related to glucose_concentration but we have even more null values for that feature

In [5]:
for nextColumn in col_names:
    fromTo = 'either' if nextColumn == 'class' else 'from'
    orTo = 'or' if nextColumn == 'class' else 'to'
    print("- {0}: {1} {2} {3} {4} ".format(nextColumn,fromTo,pima_data[nextColumn].min(),orTo,pima_data[nextColumn].max()))

- times_pregnant: from 0 to 17 
- glucose_concentration: from 44.0 to 199.0 
- blood_pressure: from 24.0 to 122.0 
- skin_fold_thickness: from 7.0 to 99.0 
- serum_insulin: from 14.0 to 846.0 
- bmi: from 18.2 to 67.1 
- pedigree: from 0.078 to 2.42 
- age: from 21 to 81 
- class: either 0 or 1 


In [6]:
pima_data.describe()

,times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,pedigree,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
pima_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 9 columns):
times_pregnant           768 non-null int64
glucose_concentration    763 non-null float64
blood_pressure           733 non-null float64
skin_fold_thickness      541 non-null float64
serum_insulin            394 non-null float64
bmi                      757 non-null float64
pedigree                 768 non-null float64
age                      768 non-null int64
class                    768 non-null int64
dtypes: float64(6), int64(3)
memory usage: 60.0 KB


In [8]:
pima_data.corr().sort('class', ascending=False)

,times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,pedigree,age,class
class,0.221898,0.494650,0.170589,0.259491,0.303454,0.313680,0.173844,0.238356,1.000000
glucose_concentration,0.128135,1.000000,0.223192,0.228043,0.581186,0.232771,0.137246,0.267136,0.494650
bmi,0.021719,0.232771,0.289230,0.648214,0.228050,1.000000,0.155382,0.025841,0.313680
serum_insulin,0.082171,0.581186,0.098272,0.184888,1.000000,0.228050,0.130395,0.220261,0.303454
skin_fold_thickness,0.100239,0.228043,0.226839,1.000000,0.184888,0.648214,0.115016,0.166816,0.259491
age,0.544341,0.267136,0.330107,0.166816,0.220261,0.025841,0.033561,1.000000,0.238356
times_pregnant,1.000000,0.128135,0.214178,0.100239,0.082171,0.021719,-0.033523,0.544341,0.221898
pedigree,-0.033523,0.137246,-0.002805,0.115016,0.130395,0.155382,1.000000,0.033561,0.173844
blood_pressure,0.214178,0.223192,1.000000,0.226839,0.098272,0.289230,-0.002805,0.330107,0.170589


### Describe the missing/NULL values. Decide if you should impute or drop them and justify your choice.

As we can see from info() below we have very FEW missing values in glucose_concentration, blood_pressure and bmi, but we have a significant number of values missing for skin_fold_thickness and serum_insulin

    DROP serum_insulin
Most medical literature seems to describe a direct relationship between insulin levels and glucose concentration in the blood in both type I and type II diabetes, therefore I am going to drop serum_insulin since it most likely will not have significant impact as a predictor

https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus&v%3Asources=medlineplus-bundle&query=Type+2+diabetes

https://www.nlm.nih.gov/medlineplus/ency/article/000305.htm

    IMPUTE skin fold thickness

After taking a quick look at some literature on BMI/SFT measurements as predictors of diabetes in adults (there is a lot more study being conducted for children and adolescents!) I have a weak idea that the verdict is out on which is a more effective predictor, they are both used, sometimes together, so I am choosing to impute skin fold thickness and fill in the missing data with the mean

In [9]:
pima_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 9 columns):
times_pregnant           768 non-null int64
glucose_concentration    763 non-null float64
blood_pressure           733 non-null float64
skin_fold_thickness      541 non-null float64
serum_insulin            394 non-null float64
bmi                      757 non-null float64
pedigree                 768 non-null float64
age                      768 non-null int64
class                    768 non-null int64
dtypes: float64(6), int64(3)
memory usage: 60.0 KB


In [19]:
pima_data.drop('serum_insulin',1, inplace=True)
pima_data
pima_data.replace({'skin_fold_thickness':{np.nan:pima_data['skin_fold_thickness'].mean()}},inplace=True)

In [21]:
pima_data.replace({'blood_pressure':{np.nan:pima_data['blood_pressure'].mean()}},inplace=True)
pima_data.replace({'bmi':{np.nan:pima_data['bmi'].mean()}},inplace=True)

### Come up with a benchmark for the minimum performance that an algorithm should have on this dataset

In [33]:
X = pima_data['class']
y = pima_data.drop('class', axis=1)
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=7)

from sklearn.dummy import DummyClassifier
dc = DummyClassifier()
dc.fit(X_train, y_train)
#dc.score(X_test, y_test) #couldnt get this to work, doing it by hand for now
pima_data['class'].value_counts() / pima_data['class'].shape[0]

0    0.651042
1    0.348958
dtype: float64

    Minimum performance is .65 which is as good as just guessing that everyone has diabetes 

### What's the best performance you can get with kNN? Is kNN a good choice for this dataset?

In [36]:
%%timeit
from sklearn.cross_validation import ShuffleSplit
cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1, weights='uniform', p=2)
knn.fit(X_train, y_train)
knn.predict(test)

from sklearn.grid_search import GridSearchCV
parameters = {'n_neighbors': range(1,30), 'weights': ['uniform', 'distance'],'p': range(1,4)}
clf = GridSearchCV(knn, parameters, cv=cv)
clf.fit(X_train, y_train)
clf.best_params_

ValueError: Found arrays with inconsistent numbers of samples: [  1 614]

In [34]:
### What's the best performance you can get with Naive Bayes? Is NB a good choice for this dataset?
### What's the best performance you can get with Logistic Regression? Is LR a good choice for this dataset?
### What's the best performance you can get with Random Forest? Is RF a good choice for this dataset?
### If you could only choose one, which classifer from the above that you already ran is best? How do you define best? (hint: could be prediction accuracy, running time, interpretability, etc)